# Run Dense reconstruction

This notebook will guide you for running a dense reconstruction with LOFTR or RoMa starting from an existing reconstruction (e.g., carried out with SuperPoint+LightGlue that are more robust for image orientation).

This notebook will perform the dense matching with RoMa and it uses pycolmap to triangulate the points from the existing camera poses using pycolmap.


In [1]:
import deep_image_matching as dim
import yaml
import pycolmap
from deep_image_matching.triangulation import db_from_existing_poses
from deep_image_matching.utils import OutputCapture

logger = dim.setup_logger("dim")

params = {
    "dir": "../assets/example_cyprus",
    "pipeline": "roma",
    "config_file": "../assets/example_cyprus/config_roma.yaml",
    "strategy": "bruteforce",
    "quality": "high",
    "tiling": "preselection",
    "skip_reconstruction": True,
    "force": True,
    "camera_options": "../assets/example_cyprus/cameras.yaml",
    "openmvg": None,
    "verbose": True,
}

# Build configuration
config = dim.Config(params)
imgs_dir = config.general["image_dir"]
output_dir = config.general["output_dir"]
config.save()

Deep Image Matching loaded in 3.552 seconds.


FileNotFoundError: Configuration file /home/francesco/phd/deep-image-matching/assets/example_cyprus/config_roma.yaml not found.

In [ ]:
# Initialize ImageMatcher class
matcher = dim.ImageMatcher(config)

# Run image matching
feature_path, match_path = matcher.run()

# Read camera options
with open(config.general["camera_options"], "r") as file:
    camera_options = yaml.safe_load(file)

# Export in colmap format
database_path = output_dir / "database.db"
dim.io.export_to_colmap(
    img_dir=imgs_dir,
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_options=camera_options,
)

In [ ]:
# Define project directory
root_path = config.general["output_dir"].parent
image_dir = config.general["image_dir"]

# Path to the pre-computed COLMAP reconstuction with the knwon poses
sfm_path = root_path / "results_superpoint+lightglue_bruteforce_quality_highest"
sfm_db_path = sfm_path / "database.db"
sfm_rec_path = sfm_path / "reconstruction"

# Path to the dense matching results to be triangulated
dense_path = config.general["output_dir"]
features_h5 = dense_path / "features.h5"
matches_h5 = dense_path / "matches.h5"
pair_file = dense_path / "pairs.txt"

# Path to the output for the dense matching
dense_db_path = dense_path / "database_dense.db"
model_path = dense_path / "dense_model"
model_path.mkdir(exist_ok=True, parents=True)

# Do geometric verification of the dense features (with known camera poses)
do_geometric_verification = True
max_error = 6
triang_min_angle = 0.1

In [ ]:
# Open sfm reconstruction with pycolmap
sfm_rec = pycolmap.Reconstruction(sfm_rec_path)

# Create a new database with the dense features and the known camera poses
db_from_existing_poses(
    dense_db_path,
    features_h5,
    matches_h5,
    sfm_rec_path,
    pair_file,
    do_geometric_verification=do_geometric_verification,
    max_error=max_error,
)

In [ ]:
# Run the triangulation with the known camera poses

# Define the options for the triangulation according to the IncrementalPipelineOptions available in pycolmap
# print(pycolmap.IncrementalPipelineOptions().summary())
opt = dict(
    triangulation=dict(
        ignore_two_view_tracks=False,
        min_angle=triang_min_angle,
    ),
)
verbose = True

with OutputCapture(verbose):
    with pycolmap.ostream():
        reconstruction = pycolmap.triangulate_points(
            sfm_rec,
            dense_db_path,
            image_dir,
            model_path,
            options=opt,
        )